In [1]:
%matplotlib inline

In [67]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from classification import *
from numpy import genfromtxt
import pywt
from kernel import *
from datetime import time, datetime, timedelta
import random
import itertools

In [3]:
import cvxopt

In [4]:
Xtr = genfromtxt('../data/Xtr.csv', delimiter=',')
Ytr = genfromtxt('../data/Ytr.csv', delimiter=',')
Xte = genfromtxt('../data/Xte.csv', delimiter=',')

Xtr = np.delete(Xtr, 3072, axis=1)
Xte = np.delete(Xte, 3072, axis=1)
Ytr = Ytr[1:,1]
N = len(Ytr)

In [5]:
def wavelet_transform(Xtr):
    result = np.zeros((Xtr.shape[0], 972))

    for i in range(Xtr.shape[0]):
        r = Xtr[i][:1024].reshape(32,32)
        g = Xtr[i][1024:2048].reshape(32,32)
        b = Xtr[i][-1024:].reshape(32,32)

        rgbArray = np.zeros((32,32,3), 'uint8')
        rgbArray[..., 0] = (r+r.min())/(r.max()-r.min())*256
        rgbArray[..., 1] = (g+g.min())/(g.max()-g.min())*256
        rgbArray[..., 2] = (b+b.min())/(b.max()-b.min())*256

        grayImage = 0.2989 *rgbArray[..., 0]+ 0.5870 *rgbArray[..., 1]+0.1140 *rgbArray[..., 2]
        coeffs2 = pywt.dwt2(grayImage, 'bior1.3')
        LL, (LH, HL, HH) = coeffs2

        result[i,:] = np.concatenate((LH.ravel(), HL.ravel(), HH.ravel()), axis=0)
    return result

In [6]:
def one_versus_all_SVM(features, labels, _lambda):
    N = len(labels)
    n_labels = len(set(labels))
    alphas = np.zeros((n_labels, N))
    bias = np.zeros(n_labels)
    
    #Linear Kernel:
    K = features.T.dot(features)
    
    for label in range(n_labels):
        one_versus_all_labels = np.zeros(N)
        for i in range(N):
            if labels[i] == label:
                one_versus_all_labels[i] = 1
            else:
                one_versus_all_labels[i] = -1
        alphas[label, :], bias[label] = train_SVM(K, one_versus_all_labels, _lambda)
        print "classifier for label ", label, " done"
        
    return alphas, bias

def predict_SVM(alphas, bias, features, X):    
    y_pred = np.zeros(alphas.shape[0])
    values_pred = np.zeros((alphas.shape[0],X.shape[1]))
    for k in range(alphas.shape[0]):
        values_pred[k,:] = alphas[k,:].dot(features.T.dot(X))+bias[k]
    return np.argmax(values_pred, axis=0)

def train_SVM(K, y, _lambda):
    
    n = y.shape[0]
    gamma = 1 / (2 * _lambda * n)
    
    P = cvxopt.matrix(K)
    
    h = cvxopt.matrix(0., (2 * n, 1))
    h[:n] = gamma
    
    A = cvxopt.matrix(1., (1, n))
    b = cvxopt.matrix(0.)
    
    y = y.astype(np.double)
    diag_y = cvxopt.spdiag(y.tolist())
    q = cvxopt.matrix(-y)
    G = cvxopt.sparse([diag_y, -diag_y])    

    res = cvxopt.solvers.qp(P, q, G, h, A, b)
    
    return np.array(res["x"]).T, res["y"][0]



## Test new SVM

In [49]:
Xtr_t = fourier_modulus_2D_kernel(Xtr)
Xte_t = fourier_modulus_2D_kernel(Xte)

In [74]:
features_functions = [fourier_modulus_1D_kernel, fourier_modulus_2D_kernel, fourier_phase_2D_kernel, wavelet_transform]
lambdas = [0.1, 0.01]

results_cross_val = {}

for feature_function in features_functions:
    Xtr_t = feature_function(Xtr)
    for _lambda in lambdas:
        results_cross_val[(feature_function, _lambda)] = []
        for i in range(5):
            mask_test = random.sample(range(0, 5000), 1000)
            mask_train =  [i for i in range(5000) if i not in mask_test]

            Xtr_train = Xtr_t[mask_train, :].T
            Xtr_test = Xtr_t[mask_test, :].T
            Ytr_train = Ytr[mask_train]
            Ytr_test = Ytr[mask_test]

            submission = False
            if submission:
                features = Xtr_t.T
                labels = Ytr
                X = Xte_t.T
            else:
                features = Xtr_train
                labels = Ytr_train
                X = Xtr_test

            t1 = datetime.now()
            alphas, bias = one_versus_all_SVM(features, labels, _lambda=_lambda)
            print 'model fitted'
            prediction = predict_SVM(alphas, bias, features, X)
            t2 = datetime.now()
            print t2-t1

            well_classified = 0
            for i in range(len(prediction)):
                if prediction[i] == Ytr_test[i]:
                     well_classified+=1
            print float(well_classified)/len(Ytr_test)
            results_cross_val[(feature_function, _lambda)].append(well_classified)

     pcost       dcost       gap    pres   dres
 0: -4.0111e+02 -5.2566e-01  3e+04  2e+02  1e-10
 1: -5.9607e+00 -4.9182e-01  4e+02  2e+00  1e-10
 2: -6.2258e-01 -8.6057e-02  3e+01  2e-01  1e-11
 3: -5.7789e-02 -5.6400e-03  2e+00  1e-02  7e-13
 4: -3.1729e-03 -1.5478e-04  1e-01  6e-04  4e-14
 5: -1.3125e-04 -3.1430e-06  4e-03  2e-05  6e-15
 6: -1.4631e-06 -4.3310e-10  4e-05  2e-07  7e-15
 7: -1.4635e-08 -4.3312e-14  4e-07  2e-09  8e-15
 8: -1.4636e-10 -4.3311e-18  4e-09  2e-11  8e-15
Optimal solution found.
classifier for label  0  done
     pcost       dcost       gap    pres   dres
 0: -3.3667e+02 -5.3724e-01  3e+04  2e+02  1e-10
 1: -5.0973e+00 -5.0294e-01  4e+02  2e+00  1e-10
 2: -5.6655e-01 -8.9126e-02  3e+01  2e-01  8e-12
 3: -5.3920e-02 -6.5838e-03  2e+00  1e-02  7e-13
 4: -3.3002e-03 -2.0099e-04  1e-01  6e-04  4e-14
 5: -1.6450e-04 -5.9203e-06  5e-03  3e-05  6e-15
 6: -1.9999e-06 -1.0445e-09  5e-05  3e-07  8e-15
 7: -2.0010e-08 -1.0446e-13  5e-07  3e-09  8e-15
 8: -2.0010e-10 -

(972, 1000)

In [75]:
with io.open('../data/cross_val_results.txt', 'w', encoding='utf-8') as f:
    f.write(unicode(json.dumps(person_id, ensure_ascii=False)))

{(<function kernel.wavelet_transform>, 0.01): [127, 113, 97, 104, 116],
 (<function kernel.wavelet_transform>, 0.1): [123, 105, 106, 113, 117],
 (<function kernel.wavelet_transform>, 1): [91, 83, 82, 93, 77],
 (<function kernel.wavelet_transform>, 10): [88, 93, 88, 88, 87],
 (<function kernel.fourier_modulus_1D_kernel>, 0.01): [272,
  261,
  262,
  257,
  266],
 (<function kernel.fourier_modulus_1D_kernel>, 0.1): [304, 310, 321, 300, 327],
 (<function kernel.fourier_modulus_1D_kernel>, 1): [81, 82, 86, 91, 94],
 (<function kernel.fourier_modulus_1D_kernel>, 10): [97, 90, 86, 88, 93],
 (<function kernel.fourier_modulus_2D_kernel>, 0.01): [278,
  290,
  271,
  281,
  289],
 (<function kernel.fourier_modulus_2D_kernel>, 0.1): [360, 328, 346, 343, 336],
 (<function kernel.fourier_modulus_2D_kernel>, 1): [90, 103, 95, 100, 97],
 (<function kernel.fourier_modulus_2D_kernel>, 10): [88, 82, 85, 94, 89],
 (<function kernel.fourier_phase_2D_kernel>, 0.01): [109, 113, 115, 120, 109],
 (<function 

In [13]:
Xte_csv = pd.read_csv('../data/Xte.csv')

In [10]:

DF = pd.DataFrame(data=prediction, columns=['Prediction'])
DF.index += 1
#DF.to_csv(filename, index=True, index_label='Id', sep=',')

## Toy data for tests

In [ ]:
n_samples = 150
mean_1 = [0, -3]
cov = [[3, 0], [0, 3]]
X_1 = np.random.multivariate_normal(mean_1, cov, n_samples).T
mean_2 = [3, 3]
X_2 = np.random.multivariate_normal(mean_2, cov, n_samples).T
mean_3 = [-3, 3]
X_3 = np.random.multivariate_normal(mean_3, cov, n_samples).T
X = np.concatenate((X_1, X_2, X_3), axis = 1)

y = np.concatenate((np.zeros((1,n_samples)), np.ones((1, n_samples)), 2*np.ones((1,n_samples))), axis=1)
y = y[0,:]
Xtr_t = X.T
Ytr = y

mask_test = range(0,450, 5)
mask_train = [i for i in range(450) if i not in mask_test]

Xtr_train = Xtr_t[mask_train, :].T
Xtr_test = Xtr_t[mask_test, :].T
Ytr_train = Ytr[mask_train]
Ytr_test = Ytr[mask_test]